In [ ]:
!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install langchain==0.1.17 --progress-bar off
!pip -qqq install llama-parse==0.1.3 --progress-bar off
!pip -qqq install qdrant-client==1.9.1  --progress-bar off
!pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
!pip -qqq install fastembed==0.2.7 --progress-bar off
!pip -qqq install flashrank==0.2.4 --progress-bar off

In [ ]:
!pip install python-dotenv

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import os

# Print the current working directory
print("Current Working Directory:", os.getcwd())

# List all files in the current directory
print("Files in the Directory:", os.listdir())

In [ ]:
from dotenv import load_dotenv

load_dotenv(".env")
groq_api_key = os.getenv("GROQ_API_KEY")

load_dotenv("llama_parse.env")
llama_parse_api_key = os.getenv("LLAMA_PARSE_API_KEY")

type(groq_api_key), type(llama_parse_api_key)
# print(groq_api_key)
# print(llama_parse_api_key)

(str, str)

In [ ]:
import os
import textwrap
from pathlib import Path

from google.colab import userdata
from IPython.display import Markdown
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse

os.environ["GROQ_API_KEY"] = groq_api_key


def print_response(response):
    response_txt = response["result"]
    for chunk in response_txt.split("\n"):
        if not chunk:
            print()
            continue
        print("\n".join(textwrap.wrap(chunk, 100, break_long_words=False)))

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import requests

file_url = "https://drive.google.com/file/d/content/citi2024prqtr2rslt.pdf/view?usp=share_link"
output_path = "/content/gdrive/MyDrive/citi_earnings.pdf"

response = requests.get(file_url)

with open(output_path, "wb") as file:
    file.write(response.content)

print(f"File downloaded to: {output_path}")

File downloaded to: /content/gdrive/MyDrive/citi_earnings.pdf


In [ ]:
instruction = """The provided document is Citi's Second Quarter 2024 Results.
This form provides detailed financial information about the company's performance for a specific quarter.
It includes unaudited financial statements, management discussion and analysis, and other relevant disclosures required by the SEC.
It contains many tables.
Try to be precise while answering the questions"""

parser = LlamaParse(
    api_key=llama_parse_api_key,
    result_type="markdown",
    parsing_instruction=instruction,
    max_timeout=5000,
)

llama_parse_documents = await parser.aload_data("/content/gdrive/MyDrive/citi2024prqtr2rslt.pdf")

Started parsing the file under job_id 82b1b030-5e0c-4965-92d2-befabf6ad430


In [ ]:
parsed_doc = llama_parse_documents[0]

In [ ]:
len(llama_parse_documents)

1

In [ ]:
Markdown(parsed_doc.text[:4096])

# For Immediate Release

# Citigroup Inc. (NYSE: C)

# July 12, 2024

# SECOND QUARTER 2024 RESULTS AND KEY METRICS

| |2Q Revenues|2Q Net Income|2Q EPS|2Q ROE|CET1 Capital Ratio|2Q RoTCE|
|---|---|---|---|---|---|---|
| |$20.1B|$3.2B|$1.52|6.3%|13.6%|7.2%(1)|

RETURNED ~$1.0 BILLION IN THE FORM OF COMMON DIVIDENDS

PAYOUT RATIO OF 34%(3)

BOOK VALUE PER SHARE OF $99.70

TANGIBLE BOOK VALUE PER SHARE OF $87.53(4)

New York, July 12, 2024 Citigroup Inc. today reported net income for the second quarter 2024 of $3.2 billion, or $1.52 per diluted share, on revenues of $20.1 billion. This compares to net income of $2.9 billion, or $1.33 per diluted share, on revenues of $19.4 billion for the second quarter 2023.

Revenues increased 4% from the prior-year period, on a reported basis. Excluding divestiture-related impacts(5), revenues were up 3%. This increase in revenues was driven by growth across all businesses, particularly in Banking, U.S. Personal Banking (USPB) and Markets. The revenue increase included an approximate $400 million gain related to the Visa B exchange completed in the second quarter 2024. A significant portion of this gain was reflected in Markets, with the remainder in All Other. The revenue growth across the businesses was partially offset by a decline in All Other.

Net income of $3.2 billion increased from $2.9 billion in the prior-year period, primarily driven by the higher revenues and lower expenses, partially offset by higher cost of credit.

Earnings per share of $1.52 increased from $1.33 per diluted share in the prior-year period, reflecting the higher net income and an approximate 1% decrease in average diluted shares outstanding.

Percentage comparisons throughout this press release are calculated for the second quarter 2024 versus the second quarter 2023, unless otherwise specified.

# CEO COMMENTARY

Results show the progress we are making in executing our strategy and the benefit of our diversified business model. We achieved positive operating leverage with revenue up 4% and a 2% decline in expenses. Services continued to grow, driven by solid fee growth, increased activity in cross border payments and new client onboardings. Markets had a strong finish to the quarter leading to better performance than we had anticipated. Fixed Income was slightly down year-over-year and Equities was up 37%, driven by strong performance in derivatives. Banking was up 38% as the wallet rebound gained some momentum and we again grew share. Wealth is starting to improve. Growth in client investment assets drove stronger investment revenue, and our focus on rationalizing the expense base is starting to pay off. U.S. Personal Banking saw revenue growth of 6%, with all three businesses again contributing to the topline.

Showcased the strength of our balance sheet. Our CET1 ratio now stands at 13.6% and we are increasing our dividend by 6%. We have made an incredible amount of progress in simplification - both strategically and organizationally. We are modernizing our infrastructure to improve our client service and automating processes to strengthen controls. We will continue to execute our transformation and our strategy so we can meet our medium-term targets and continue to further improve our returns over.
---
# Second Quarter Financial Results

|Citigroup|2Q'24|1Q'24|2Q'23|QoQ%|YoY%|
|---|---|---|---|---|---|
|Total revenues, net of interest expense|20,139|21,104|19,436|(5)%|4%|
|Total operating expenses|13,353|14,195|13,570|(6)%|(2)%|
|Net credit losses|2,283|2,303|1,504|(1)%|52%|
|Net ACL build / (release)(a)|68|21|161|NM|(58)%|
|Other provisions(b)|125|41|159|NM|(21)%|
|Total cost of credit|2,476|2,365|1,824|5%|36%|
|Income (loss) from continuing operations before taxes|4,310|4,544|4,042|(5)%|7%|
|Provision for income taxes|1,047|1,136|1,090|(8)%|(4)%|
|Income (loss) from continuing operations|3,263|3,408|2,952|(4)%|11%|
|Income (loss) from discontinued operations, net of taxes|-|(1)|(1)|100%|100%|
|Net income attributable to non-controlling interest|46|36|36|28%|28%|
|Citigroup's net income (loss)|$3,21

In [ ]:
data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [ ]:
document_path = Path("data/parsed_document.md")
if not document_path.exists():
  document_path.touch()
with document_path.open("a") as f:
  f.write(parsed_doc.text)

In [ ]:
loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
type(document_path), type(loaded_documents)

(pathlib.PosixPath, list)

In [ ]:
len(loaded_documents)

1

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=128)
docs = text_splitter.split_documents(loaded_documents)
len(docs)

22

In [ ]:
docs[0]

Document(page_content='For Immediate Release\n\nCitigroup Inc. (NYSE: C)\n\nJuly 12, 2024\n\nSECOND QUARTER 2024 RESULTS AND KEY METRICS\n\n2Q Revenues 2Q Net Income 2Q EPS 2Q ROE CET1 Capital Ratio 2Q RoTCE $20.1B $3.2B $1.52 6.3% 13.6% 7.2%(1)\n\nRETURNED ~$1.0 BILLION IN THE FORM OF COMMON DIVIDENDS\n\nPAYOUT RATIO OF 34%(3)\n\nBOOK VALUE PER SHARE OF $99.70\n\nTANGIBLE BOOK VALUE PER SHARE OF $87.53(4)\n\nNew York, July 12, 2024 Citigroup Inc. today reported net income for the second quarter 2024 of $3.2 billion, or $1.52 per diluted share, on revenues of $20.1 billion. This compares to net income of $2.9 billion, or $1.33 per diluted share, on revenues of $19.4 billion for the second quarter 2023.\n\nRevenues increased 4% from the prior-year period, on a reported basis. Excluding divestiture-related impacts(5), revenues were up 3%. This increase in revenues was driven by growth across all businesses, particularly in Banking, U.S. Personal Banking (USPB) and Markets. The revenue in

In [ ]:
print(docs[0].page_content)

For Immediate Release

Citigroup Inc. (NYSE: C)

July 12, 2024

SECOND QUARTER 2024 RESULTS AND KEY METRICS

2Q Revenues 2Q Net Income 2Q EPS 2Q ROE CET1 Capital Ratio 2Q RoTCE $20.1B $3.2B $1.52 6.3% 13.6% 7.2%(1)

RETURNED ~$1.0 BILLION IN THE FORM OF COMMON DIVIDENDS

PAYOUT RATIO OF 34%(3)

BOOK VALUE PER SHARE OF $99.70

TANGIBLE BOOK VALUE PER SHARE OF $87.53(4)

New York, July 12, 2024 Citigroup Inc. today reported net income for the second quarter 2024 of $3.2 billion, or $1.52 per diluted share, on revenues of $20.1 billion. This compares to net income of $2.9 billion, or $1.33 per diluted share, on revenues of $19.4 billion for the second quarter 2023.

Revenues increased 4% from the prior-year period, on a reported basis. Excluding divestiture-related impacts(5), revenues were up 3%. This increase in revenues was driven by growth across all businesses, particularly in Banking, U.S. Personal Banking (USPB) and Markets. The revenue increase included an approximate $400 million

In [ ]:
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/218M [00:00<?, ?B/s]

In [ ]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    # location=":memory:",
    path="./db",
    collection_name="document_embeddings",
)

In [ ]:
%%time
query = "What is the most important update from Citi's earnings?"
similar_docs = qdrant.similarity_search_with_score(query)

CPU times: user 294 ms, sys: 0 ns, total: 294 ms
Wall time: 293 ms


In [ ]:
# type(similar_docs)
print(len(similar_docs))
similar_docs[0]

4


(Document(page_content='For Immediate Release\n\nCitigroup Inc. (NYSE: C)\n\nJuly 12, 2024\n\nSECOND QUARTER 2024 RESULTS AND KEY METRICS\n\n2Q Revenues 2Q Net Income 2Q EPS 2Q ROE CET1 Capital Ratio 2Q RoTCE $20.1B $3.2B $1.52 6.3% 13.6% 7.2%(1)\n\nRETURNED ~$1.0 BILLION IN THE FORM OF COMMON DIVIDENDS\n\nPAYOUT RATIO OF 34%(3)\n\nBOOK VALUE PER SHARE OF $99.70\n\nTANGIBLE BOOK VALUE PER SHARE OF $87.53(4)\n\nNew York, July 12, 2024 Citigroup Inc. today reported net income for the second quarter 2024 of $3.2 billion, or $1.52 per diluted share, on revenues of $20.1 billion. This compares to net income of $2.9 billion, or $1.33 per diluted share, on revenues of $19.4 billion for the second quarter 2023.\n\nRevenues increased 4% from the prior-year period, on a reported basis. Excluding divestiture-related impacts(5), revenues were up 3%. This increase in revenues was driven by growth across all businesses, particularly in Banking, U.S. Personal Banking (USPB) and Markets. The revenue i

In [ ]:
(similar_docs[0][0].page_content)

'For Immediate Release\n\nCitigroup Inc. (NYSE: C)\n\nJuly 12, 2024\n\nSECOND QUARTER 2024 RESULTS AND KEY METRICS\n\n2Q Revenues 2Q Net Income 2Q EPS 2Q ROE CET1 Capital Ratio 2Q RoTCE $20.1B $3.2B $1.52 6.3% 13.6% 7.2%(1)\n\nRETURNED ~$1.0 BILLION IN THE FORM OF COMMON DIVIDENDS\n\nPAYOUT RATIO OF 34%(3)\n\nBOOK VALUE PER SHARE OF $99.70\n\nTANGIBLE BOOK VALUE PER SHARE OF $87.53(4)\n\nNew York, July 12, 2024 Citigroup Inc. today reported net income for the second quarter 2024 of $3.2 billion, or $1.52 per diluted share, on revenues of $20.1 billion. This compares to net income of $2.9 billion, or $1.33 per diluted share, on revenues of $19.4 billion for the second quarter 2023.\n\nRevenues increased 4% from the prior-year period, on a reported basis. Excluding divestiture-related impacts(5), revenues were up 3%. This increase in revenues was driven by growth across all businesses, particularly in Banking, U.S. Personal Banking (USPB) and Markets. The revenue increase included an app

In [ ]:
for doc, score in similar_docs:
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {score}")
    print("-" * 80)
    print()

text: For Immediate Release

Citigroup Inc. (NYSE: C)

July 12, 2024

SECOND QUARTER 2024 RESULTS AND KEY METRICS

2Q Revenues 2Q Net Income 2Q EPS 2Q ROE CET1 Capital Ratio 2Q RoTCE $20.1B $3.2B $1.52 6.3% 13.6% 7.2%(1)

RETURNED ~$1.0 BILLION IN THE FORM OF CO

score: 0.7038000839142019
--------------------------------------------------------------------------------

text: (b) Includes provisions on Other Assets, policyholder benefits and claims and HTM debt securities.

Citigroup

Citigroup revenues of $20.1 billion in the second quarter 2024 increased 4%, on a reported basis. Excluding divestiture-related impacts(5), reven

score: 0.6678057306390563
--------------------------------------------------------------------------------

text: Wealth at Work revenues of $195 million decreased 13%, driven by deposit spread compression and higher mortgage funding costs, partially offset by higher investment fee revenues.

Citigold revenues of $1.0 billion increased 6%, driven by higher inves

In [ ]:
%%time
retriever = qdrant.as_retriever(search_kwargs={"k": 5})
retrieved_docs = retriever.invoke(query)

CPU times: user 290 ms, sys: 0 ns, total: 290 ms
Wall time: 291 ms


In [ ]:
# len(retrieved_docs)
retrieved_docs[0].page_content
# retrieved_docs[0].metadata

'For Immediate Release\n\nCitigroup Inc. (NYSE: C)\n\nJuly 12, 2024\n\nSECOND QUARTER 2024 RESULTS AND KEY METRICS\n\n2Q Revenues 2Q Net Income 2Q EPS 2Q ROE CET1 Capital Ratio 2Q RoTCE $20.1B $3.2B $1.52 6.3% 13.6% 7.2%(1)\n\nRETURNED ~$1.0 BILLION IN THE FORM OF COMMON DIVIDENDS\n\nPAYOUT RATIO OF 34%(3)\n\nBOOK VALUE PER SHARE OF $99.70\n\nTANGIBLE BOOK VALUE PER SHARE OF $87.53(4)\n\nNew York, July 12, 2024 Citigroup Inc. today reported net income for the second quarter 2024 of $3.2 billion, or $1.52 per diluted share, on revenues of $20.1 billion. This compares to net income of $2.9 billion, or $1.33 per diluted share, on revenues of $19.4 billion for the second quarter 2023.\n\nRevenues increased 4% from the prior-year period, on a reported basis. Excluding divestiture-related impacts(5), revenues were up 3%. This increase in revenues was driven by growth across all businesses, particularly in Banking, U.S. Personal Banking (USPB) and Markets. The revenue increase included an app

In [ ]:
for doc in retrieved_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print("-" * 80)
    print()

id: e5036fcc18e94586ae053a62f7dcadc7

text: For Immediate Release

Citigroup Inc. (NYSE: C)

July 12, 2024

SECOND QUARTER 2024 RESULTS AND KEY METRICS

2Q Revenues 2Q Net Income 2Q EPS 2Q ROE CET1 Capital Ratio 2Q RoTCE $20.1B $3.2B $1.52 6.3% 13.6% 7.2%(1)

RETURNED ~$1.0 BILLION IN THE FORM OF CO

--------------------------------------------------------------------------------

id: 288fcdfd5b7143eea1188d8e11370e3d

text: (b) Includes provisions on Other Assets, policyholder benefits and claims and HTM debt securities.

Citigroup

Citigroup revenues of $20.1 billion in the second quarter 2024 increased 4%, on a reported basis. Excluding divestiture-related impacts(5), reven

--------------------------------------------------------------------------------

id: 65cdb42c71824a8abb0e296e4b26fcfb

text: Wealth at Work revenues of $195 million decreased 13%, driven by deposit spread compression and higher mortgage funding costs, partially offset by higher investment fee revenues.

Citigold

In [ ]:
compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

ms-marco-MiniLM-L-12-v2.zip: 100%|██████████| 21.6M/21.6M [00:00<00:00, 68.0MiB/s]


In [ ]:
%%time
reranked_docs = compression_retriever.invoke(query)
len(reranked_docs)

Running pairwise ranking..
CPU times: user 2.19 s, sys: 205 ms, total: 2.4 s
Wall time: 2.28 s


3

In [ ]:
for doc in reranked_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {doc.metadata['relevance_score']}")
    print("-" * 80)
    print()

id: e5036fcc18e94586ae053a62f7dcadc7

text: For Immediate Release

Citigroup Inc. (NYSE: C)

July 12, 2024

SECOND QUARTER 2024 RESULTS AND KEY METRICS

2Q Revenues 2Q Net Income 2Q EPS 2Q ROE CET1 Capital Ratio 2Q RoTCE $20.1B $3.2B $1.52 6.3% 13.6% 7.2%(1)

RETURNED ~$1.0 BILLION IN THE FORM OF CO

score: 0.40258365869522095
--------------------------------------------------------------------------------

id: 288fcdfd5b7143eea1188d8e11370e3d

text: (b) Includes provisions on Other Assets, policyholder benefits and claims and HTM debt securities.

Citigroup

Citigroup revenues of $20.1 billion in the second quarter 2024 increased 4%, on a reported basis. Excluding divestiture-related impacts(5), reven

score: 0.19538599252700806
--------------------------------------------------------------------------------

id: 5050eb15c998483aad1c993a4af22ec2

text: Facebook: www.facebook.com/citi

LinkedIn: www.linkedin.com/company/citi

Forward-Looking Statements

Certain statements in this relea

In [ ]:
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

In [ ]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": True},
)

In [ ]:
%%time
response = qa.invoke(query)

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: For Immediate Release

Citigroup Inc. (NYSE: C)

July 12, 2024

SECOND QUARTER 2024 RESULTS AND KEY METRICS

2Q Revenues 2Q Net Income 2Q EPS 2Q ROE CET1 Capital Ratio 2Q RoTCE $20.1B $3.2B $1.52 6.3% 13.6% 7.2%(1)

RETURNED ~$1.0 BILLION IN THE FORM OF COMMON DIVIDENDS

PAYOUT RATIO OF 34%(3)

BOOK VALUE PER SHARE OF $99.70

TANGIBLE BOOK VALUE PER SHARE OF $87.53(4)

New York, July 12, 2024 Citigroup Inc. today reported net income for the second quarter 2024 of $3.2 billion, or $1.52 per diluted share, on revenues of $20.1 billion. This compares to net income of $2.9 billion, or $1.33 per diluted share, on revenues of $19.4 billion for the second quarter 2023.

Revenues increased 4%

In [ ]:
response.keys()

dict_keys(['query', 'result', 'source_documents'])

In [ ]:
print_response(response)

**Answer:** The most important update from Citi's earnings is the 4% increase in revenues to $20.1
billion, driven by growth across all businesses, particularly in Banking, U.S. Personal Banking
(USPB), and Markets.

**Additional helpful information:**

* Net income increased to $3.2 billion, or $1.52 per diluted share, compared to $2.9 billion, or
$1.33 per diluted share, in the prior-year period.
* The revenue growth was partially offset by a decline in All Other, and included an approximate
$400 million gain related to the Visa B exchange completed in the second quarter 2024.
* Citigroup returned ~$1.0 billion in the form of common dividends, with a payout ratio of 34%.


In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": False},
)

In [ ]:
# %%time
# response = qa.invoke("How did Citi's 2nd Quarter 2024 earnings compare to the 1st Quarter?")
# Markdown(response["result"])

In [ ]:
response = qa.invoke("What is the most important update from Citi's Q2 earnings?")
Markdown(response["result"])

Running pairwise ranking..


**Most Important Update from Citi's Q2 Earnings:**

The most important update from Citi's Q2 earnings is the 4% increase in revenues to $20.1 billion, driven by growth across all businesses, particularly in Banking, U.S. Personal Banking (USPB), and Markets.

**Additional Helpful Information:**

* Net income increased to $3.2 billion, or $1.52 per diluted share, compared to $2.9 billion, or $1.33 per diluted share, in the prior-year period.
* The company returned ~$1.0 billion in the form of common dividends, with a payout ratio of 34%.
* Book value per share increased to $99.70, and tangible book value per share rose to $87.53.

In [ ]:
response = qa.invoke("How does Citi make money?")
Markdown(response["result"])

Running pairwise ranking..


**How does Citi make money?**

Citi makes money by providing a broad range of financial products and services to corporations, governments, investors, institutions, and individuals across nearly 180 countries and jurisdictions. The company generates revenue through various business segments, including Banking, U.S. Personal Banking (USPB), Markets, and All Other.

**Additional helpful information:**

* Citi's revenue for the second quarter 2024 was $20.1 billion, with a 4% increase from the prior-year period.
* The revenue growth was driven by growth across all businesses, particularly in Banking, USPB, and Markets.
* The company also reported a gain of approximately $400 million related to the Visa B exchange completed in the second quarter 2024.

Note: The provided information does not provide a detailed breakdown of Citi's revenue streams, but it gives an overview of the company's business segments and revenue growth.

In [ ]:
response = qa.invoke("What was the main driver of the 4% increase in revenue?")
Markdown(response["result"])

Running pairwise ranking..


**Answer:** The main driver of the 4% increase in revenue was growth across all businesses, particularly in Banking, USPB, and Markets.

**Additional helpful information:**

* The revenue increase included an approximate $400 million gain related to the Visa B exchange completed in the second quarter 2024, which was reflected in Markets and All Other.
* The growth in revenue was partially offset by a decline in All Other.

In [ ]:
response = qa.invoke("What do you mean by 'a decline in All Other'?")
Markdown(response["result"])

Running pairwise ranking..


**Answer:** 

By 'a decline in All Other', it refers to the 7% decrease in All Other (managed basis) expenses of $2.1 billion, primarily driven by the closed exits and wind-downs, partially offset by the civil money penalties imposed by the FRB and the OCC.

**Additional Information:**

* All Other (managed basis) includes Legacy Franchises and certain unallocated costs of global staff functions, other corporate expenses, and unallocated global operations and technology expenses and income taxes, as well as Corporate Treasury investment activities and discontinued operations.
* The decline in expenses was partially offset by higher cost of credit, which increased to $243 million, driven by a higher ACL build and higher net credit losses.

In [ ]:
response = qa.invoke("Can you calculate Citi's EBITDA in Q2?")
Markdown(response["result"])

Running pairwise ranking..


**EBITDA Calculation:**

Unfortunately, the provided information does not explicitly mention EBITDA (Earnings Before Interest, Taxes, Depreciation, and Amortization). However, we can try to estimate it using the available data.

From the income statement, we have:

* Net income: $3,217 million
* Provision for income taxes: $1,047 million
* Income from continuing operations before taxes: $4,310 million
* Total operating expenses: $13,353 million

We can estimate EBITDA by adding back depreciation and amortization to income from continuing operations before taxes. Unfortunately, the exact depreciation and amortization figures are not provided. However, we can make an educated estimate based on the industry average or historical data.

Assuming a depreciation and amortization expense of around 10% of total operating expenses (a rough estimate), we get:

* Depreciation and amortization: $1,335 million (10% of $13,353 million)

Now, let's calculate EBITDA:

* EBITDA ≈ Income from continuing operations before taxes + Depreciation and amortization
* EBITDA ≈ $4,310 million + $1,335 million
* EBITDA ≈ $5,645 million

Please note that this is a rough estimate and the actual EBITDA figure might be different.

**Additional Information:**

* Citigroup's net income increased by 10% year-over-year, driven by higher revenues and lower expenses, partially offset by higher cost of credit.
* The company returned approximately $1.0 billion in the form of common dividends, with a payout ratio of 34%.
* Book value per share and tangible book value per share increased by 2% and 3%, respectively, year-over-year.

In [ ]:
response = qa.invoke("What does the CEO Commentary mention?")
Markdown(response["result"])

Running pairwise ranking..


**CEO Commentary Mentions:**

The CEO Commentary mentions the progress made in executing the company's strategy and the benefits of its diversified business model. It highlights the positive operating leverage, with revenue up 4% and a 2% decline in expenses. The commentary also mentions the strong performance of various business segments, including Services, Markets, Fixed Income, Equities, Banking, and Wealth. Additionally, it notes the strength of the company's balance sheet, with a CET1 ratio of 13.6%, and the decision to increase the dividend by 6%.

**Additional Helpful Information:**

* The company has made significant progress in simplification, both strategically and organizationally, and is modernizing its infrastructure to improve client service and automate processes to strengthen controls.
* The company is focused on executing its transformation and strategy to meet its medium-term targets and continue to improve returns.

In [ ]:
response = qa.invoke("What should I be looking for specifically about the provided financial information?")
Markdown(response["result"])

Running pairwise ranking..


**What to look for in the provided financial information:**

1. **Revenue growth**: Look for changes in revenue across different segments, such as Services, Markets, and others. Identify areas of growth and decline.
2. **Net income and profitability**: Analyze the net income and profitability metrics, such as Return on Tangible Common Equity (RoTCE), to understand the company's ability to generate earnings.
3. **Expenses and cost management**: Review total operating expenses and identify areas of increase or decrease. Assess the company's ability to manage costs.
4. **Credit quality and provisioning**: Examine net credit losses, provision for credit losses, and the allowance for credit losses to understand the company's credit risk management.
5. **Capital adequacy and regulatory metrics**: Review the company's capital position, including Common Equity Tier 1 Capital (CET1), Total Tier 1 Capital (T1C), and the Supplementary Leverage Ratio (SLR), to assess its ability to meet regulatory requirements.
6. **Segment performance**: Analyze the performance of individual segments, such as Treasury and Trade Solutions, Securities Services, and Markets, to identify areas of strength and weakness.

**Additional helpful information:**

* The company provides a range of financial metrics, including non-GAAP measures, to help investors and analysts understand its performance.
* The Appendices provide additional details on specific areas, such as capital adequacy, tangible common equity, and regulatory metrics.
* The company's website (www.citigroup.com) offers more information on its business, financials, and segment trends.

In [ ]:
response = qa.invoke("What was the difference between investment banking and corporate lending sales in Q2 2024?")
Markdown(response["result"])

Running pairwise ranking..


**Answer:** The difference between investment banking and corporate lending sales in Q2 2024 is $88 million ($853 million in investment banking vs. $765 million in corporate lending).

**Additional helpful information:**

* Investment banking revenues increased by 60% year-over-year, while corporate lending revenues increased by 7% year-over-year.
* Total Banking revenues, including gain/loss on loan hedges, were $1,627 million in Q2 2024, with a 38% year-over-year increase.